## Notebook to explain an individual with different backgrounds

In [1]:
import xgboost, shap, os, scipy, sklearn, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
sys.path.append("/homes/gws/hughchen/deepshap-exploration/")

# Load data
from deepshap.data import load_nhanes, get_nhanes_feat_names
trainx, trainy, _, _, testx, testy = load_nhanes()
feat_names = get_nhanes_feat_names()

# Load model
from model import nhanes_mlp
model = nhanes_mlp()

Using TensorFlow backend.


KeyboardInterrupt: 

### Explain a single older male sample

In [ ]:
from sklearn.model_selection import train_test_split
from deepshap.data import preprocessed_data

# Get the raw un-standardized data
X, y = preprocessed_data()
pids = np.unique(X.index.values)
train_pids,test_pids = train_test_split(pids, random_state=0)
strain_pids,valid_pids = train_test_split(train_pids, random_state=0)
strain_inds = np.where([p in strain_pids for p in X.index.values])[0]
raw_trainx = X.iloc[strain_inds,:]

In [ ]:
shap.initjs()
# Get older male data set
older_male_inds = (raw_trainx["sex_isFemale"] == False) & (raw_trainx["age"] > 60)
trainx_older_males = trainx[older_male_inds]
raw_trainx_older_males = raw_trainx[older_male_inds]

# Older male sample to explain
expli_ind = 5
explicand = trainx_older_males[expli_ind:expli_ind+1,:]
explicand_raw = raw_trainx_older_males.iloc[expli_ind]
print(explicand_raw)

is_save = True
# is_save = False

#####################################################
### Explain with background of random individuals ###
#####################################################
np.random.seed(10)
n = trainx_older_males.shape[0]; stepsize = 1000
refe = trainx[np.random.choice(n,stepsize,replace=False)]
expl = shap.DeepExplainer(model, refe)
attr = expl.shap_values(explicand)[0]

# Make and save force plot
if is_save:
    fig = shap.force_plot(expl.expected_value, attr, explicand_raw, 
                          feat_names, matplotlib=True, show=False)
    fig.savefig("fig/ind{}_randbackground.pdf".format(expli_ind))
else:
    shap.force_plot(expl.expected_value, attr, explicand_raw, feat_names, matplotlib=True)
    
#####################################################
### Explain with background of random older males ###
#####################################################
# Explain with background of random older males
np.random.seed(10)
n = trainx_older_males.shape[0]; stepsize = 1000
inds = np.random.choice(n,stepsize,replace=False)
refe = trainx_older_males[inds]
expl = shap.DeepExplainer(model, refe)
attr = expl.shap_values(explicand)[0]

# Make and save force plot
if is_save:
    fig = shap.force_plot(expl.expected_value, attr, explicand_raw, 
                          feat_names, matplotlib=True, show=False)
    fig.savefig("fig/ind{}_oldmalebackground.pdf".format(expli_ind))
else:
    shap.force_plot(expl.expected_value, attr, explicand_raw, feat_names, matplotlib=True)